In [3]:
%pip install -q -U kaggle_environments


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import random
import math

from kaggle_environments import make

2024-12-08 12:59:49.155835: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


No pygame installed, ignoring import


Опишем поведение агента, всегда играющего "камень" - это значение 0

In [5]:
def rock_agent():
  return 0

Опишем поведение агента, всегда играющего "бумагу" - это значение 1

In [6]:
def paper_agent():
  return 1

Опишем поведение агента, всегда играющего "ножницы" - это значение 2

In [7]:
def scissors_agent():
  return 2

Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [8]:
def copy_agent(observation):
  # в случае, когда предыдущий ход известен используем его
  if observation.step > 0:
    return observation.lastOpponentAction
  # первый раз выбираем случайных ход
  else:
    return random.choice([0, 1, 2])

Опишем агента, который выбирает ход случайным образом

In [9]:
def random_agent():
  return random.choice([0, 1, 2])

Опишем агента, который выбирает ход случайным образом с предпочтением хода "Ножницы" - для этого присваеваеваем каждому ходу вес: 20% для "камня", 20% для "бумаги" и 60% для "ножниц"

In [10]:
def weighted_random_agent():
  return random.choices(population=[0, 1, 2], cum_weights=(10, 10, 80), k=1)[0]

Опишем агента, который последовательно выбирает ход по кругу: "камень", "бумага", "ножницы", "камень" и т.д.

In [11]:
def cycling_agent(observation):
  return [0, 1, 2][observation.step % 3]

Опишем агента, который последовательно каждые три шага выбирает новый ход по кругу

In [13]:
def cycling_3_agent(observation):
  return [0, 1, 2][(observation.step // 3) % 3]

Опишем агента, который повторяет последний успешный ход. Если сыграли в ничью или был проигрыш - выбирает новый ход

In [14]:
# вспомогательная функция, которая определяет результат хода для игрока left_move
def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )

    # возвращает 0, если сыграли в ничью, 1 если победил left_move, -1 если left_move проиграл
    return 0 if delta == 0 else math.copysign(1, delta)


last_action = None

def repeat_success_agent(observation):
    global last_action

    if observation.step == 0 or get_score(last_action, observation.lastOpponentAction) <= 0:
        last_action = random.choice([0, 1, 2])

    return last_action

Опишем агента, который выбирает случайный ход, но не такой как в прошлый раз

In [19]:
last_action = None

# non-repeating random

# Агент выбирает случайный ход, но не такой как в прошлый раз
def random_skip_last_agent(observation):
    global last_action

    if observation.step == 0:
      # на первом шаге выбираем случайным образом любой из 3 доступных ходов
      last_action = random.choice([0, 1, 2])
    else:
      # со второго шага исключаем предыдущий ход из выборки
      avaliable_actions = list(filter(lambda action: action != last_action, [0, 1, 2]))
      # выбираем случайным образом любой из 2 оставшихся
      last_action = random.choice(avaliable_actions)

    return last_action


Опишем агента, который делает два хода "ножницами", а затем всегда играет "бумагой"

In [16]:
def double_scissors_then_paper_agent(observation):
    if observation.step % 3 < 2:
        return 2
    return 1

Опишем агента, который делает первый ход случайным образом, а затем выбирает ход проигрышный по отношению к предыдущему ходу противника

In [18]:
def previous_lose_agent(observation):
  if observation.step > 0:
    if observation.lastOpponentAction == 0:
      return 2
    elif observation.lastOpponentAction == 1:
      return 0
    elif observation.lastOpponentAction == 2:
      return 1
  else:
    return random.choice([0, 1, 2])

In [20]:
agents = {
    'rock': rock_agent,
    'paper': paper_agent,
    'scissors': scissors_agent,
    'copy': copy_agent,
    'random': random_agent,
    'weighted_random': weighted_random_agent,
    'cycling': cycling_agent,
    'cycling_3': cycling_3_agent,
    'repeat_success': repeat_success_agent,
    'random_skip_last': random_skip_last_agent,
    'double_scissors_then': double_scissors_then_paper_agent,
    'previous_lose': previous_lose_agent
}

Опишем функцию для запуска турнира между агентами

In [21]:
def run_tournament(agents):
    environment = make("rps", configuration={"episodeSteps": 100})
    result = {name: 0 for name in agents}

    # каждый агент играет с каждым
    for left_name, left_agent in agents.items():
        for right_name, right_agent in agents.items():
            # aгент не играет сам с собой
            if left_name == right_name:
                continue

            environment.reset()
            # запуск игры между "левым" и "правым" агентами
            outcome = environment.run([left_agent, right_agent])
            final_state = outcome[-1]

            left_reward = final_state[0]["reward"]
            right_reward = final_state[1]["reward"]

            # обновляем счет
            if left_reward > right_reward:
                result[left_name] += 1
            elif right_reward > left_reward:
                result[right_name] += 1

    return result

Запустим турнир и выведем результаты в логи

In [22]:

tournament_results = run_tournament(agents)
sorted_result = sorted(tournament_results.items(), key=lambda x: x[1], reverse=True)

print("Результат турнира:")
for agent, score in sorted_result:
    print(f"{agent}: {score} побед")

Результат турнира:
repeat_success: 11 побед
rock: 8 побед
scissors: 6 побед
weighted_random: 6 побед
previous_lose: 5 побед
paper: 4 побед
cycling: 4 побед
cycling_3: 4 побед
double_scissors_then: 2 побед
copy: 1 побед
random: 0 побед
random_skip_last: 0 побед
